In [1]:
import os
import tarfile
import urllib

In [2]:
TYPE = "tgz"
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/refs/heads/master/"
HOUSING_PATH = os.path.join('dataset', 'housing')
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing." + TYPE
print(HOUSING_URL)

https://raw.githubusercontent.com/ageron/handson-ml2/refs/heads/master/datasets/housing/housing.tgz


In [3]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH, type=TYPE):
    os.makedirs(housing_path, exist_ok=True)
    if type == "tgz":
        print("teste")
        download_path = os.path.join(housing_path, 'housing.tgz')
    elif type == "csv":
        download_path = os.path.join(housing_path, 'housing.csv')
    else:
        print("Tipo não encontrado")
        return
    urllib.request.urlretrieve(housing_url, download_path)
    if type == 'tgz':
        housing_tgz = tarfile.open(download_path)
        housing_tgz.extractall(path=housing_path)
        housing_tgz.close()


In [4]:
import pandas as pd

In [5]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [6]:
fetch_housing_data()

teste


In [9]:
df = load_housing_data()
df.shape

(20640, 10)